# 🧠 CrewAI: Multi-Agent Bidding Example

In this notebook, you'll define a team of agents who will **bid** on tasks based on their roles and assign the most suitable agent to each task dynamically.


## ⚙️ Step 1: Environment Setup
This installs and runs the `ollama` backend and exposes the service via localtunnel tunnel. Make sure to:
- Restart the runtime if needed
- Use the ngrok alternative (if Cloudflare is blocked or throttled)

In [ ]:
%pip install ollama chromadb
%pip install colab-xterm

## 🛠️ System Info Tools (Optional)

Installs utilities (`pciutils`, `lshw`) to inspect hardware specs — useful for checking GPU/CPU availability in Colab or local runtime.








In [ ]:
!sudo apt-get update
!sudo apt-get install pciutils lshw

## 📦 Ollama Installation

Downloads and installs Ollama via the official shell script — run this once per environment setup.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

## 🔧 Step 2: Programmatic Model Management and Server Initialization

In this section, we:
- Import the required libraries for managing subprocesses, HTTP requests, and multithreading
- Start the Ollama server programmatically using a background thread
- Pull the required models (`deepseek-r1:7b`, `llama3`) using `ollama pull`
- Optionally include fallback to a smaller model (`deepseek-r1:1.5b`)
- Confirm the list of available models and test that the local Ollama server is running at `localhost:11434`

📌 **Why it matters**: This sets up your local model infrastructure for agent interaction. You'll later reference `localhost:11434` in your agent definitions to connect to these models.


In [ ]:
# Import necessary libraries
import subprocess
import requests
import json
import threading
from pprint import pprint

##  Launching the Ollama Server in Background

Before using any model, we need to start the **Ollama inference server**, which listens by default on `localhost:11434`.

This snippet:
- Defines a Python function `run_ollama()` that launches `ollama serve`
- Starts it in a **background thread**, so the notebook remains interactive
- Allows the server to stay active without blocking further cells

🛠️ **Note**: You only need to run this once per session. If you restart your Colab, re-run this cell before using any models.


In [ ]:
# Start the Ollama server
def run_ollama():
  subprocess.Popen(["ollama", "serve"])
thread = threading.Thread(target=run_ollama)
thread.start()

## 📥 Pulling Models

We download pre-trained models from the Ollama registry:
- `deepseek-r1:7b` – reasoning & code
- `llama3` – general-purpose assistant

In [ ]:
# Download the deepseek-r1:7b distilled model
!ollama pull deepseek-r1:7b
#!ollama pull llama3
!ollama pull deepseek-coder:6.7b
# If this doesn't work, you can uncomment the below code to download a smaller model- deepseek-r1:1.5b
# !ollama pull deepseek-r1:1.5b

## 🪶 Pulling Lightweight SLMs

These small models are ideal for fast local agents and low-resource environments:
- `phi3:mini`, `tinyllama` – ultra-small general models
- `gemma:2b` – Google's compact chat model
- `deepseek-r1:1.5b` – distilled reasoning model

In [ ]:
#!ollama pull phi3:mini
#!ollama pull tinyllama
#!ollama pull gemma:2b
!ollama pull deepseek-r1:1.5b

### 🧠 Downloading Embedding Models

To enable **Retrieval-Augmented Generation (RAG)**, we need to convert documents and queries into **vector embeddings**. These vectors help the agent understand meaning and similarity between texts.

The following commands pull different embedding models using **Ollama**, which we’ll use later to build a vectorstore for document search.



In [ ]:
!ollama pull mxbai-embed-large:latest
#!ollama pull nomic-embed-text
#!ollama pull all-minilm

## 🔌 Test Ollama Server

Sends a test request to verify the Ollama server is running on `localhost:11434`.

In [ ]:
!curl http://127.0.0.1:11434

## 📄 Check Installed Models

Lists all models currently downloaded and available in your local Ollama environment.

In [ ]:
!ollama list

### 📚 Embedding Documents into a Vector Store

In this step, we convert a list of simple documents (facts about llamas) into **vector embeddings** and store them in a searchable database using `Chroma`.

We'll use **Ollama** to generate embeddings with a local model (`mxbai-embed-large`), and **ChromaDB** to store and later retrieve these vectors.


In [ ]:
import ollama
import chromadb

# 🧠 Agent roles + process knowledge + product information + pricing
documents = [
  # Agent definitions
  "The SalesRep is responsible for initiating conversations with customers, discovering their needs, and building trust through dialogue.",
  "The ProductRecommender analyzes customer preferences and suggests the most appropriate products from available stock.",
  "The QuotationAgent prepares clear, itemized quotes that include discounts, taxes, and delivery options.",
  "The CRMUpdater logs customer details, preferences, and quote history into the CRM for future reference and follow-ups.",

  # Retail process knowledge
  "In retail, a good sales conversation starts by understanding the customer's needs through qualifying questions.",
  "Product recommendations should be based on customer lifestyle, preferences, and intended use.",
  "A quote must include item breakdowns, applied discounts, taxes, and any relevant shipping or warranty info.",
  "CRM systems are essential for tracking interactions, updating contact info, and managing follow-up tasks.",
  "Follow-up messages after a quote can increase conversion rates by over 60%, especially if personalized.",
  "New customers are often offered a 10% discount as part of welcome campaigns in retail.",
  "Retail agents often work as a team: a sales rep engages the client, a recommender suggests items, and another agent prepares the invoice.",

  # Product info + pricing
  "The MountainFlow 45L Backpack is waterproof, lightweight, and perfect for 2–5 day hikes or urban travel. Price: $89.99",
  "The CloudShell Rain Jacket is windproof, breathable, and designed for sudden weather changes. Price: $129.00",
  "PowerCore Mini Charger provides 5200mAh backup power, USB-C support, and airplane-safe batteries. Price: $29.95",
  "The TrailMax Pro Hiking Shoes are made for long treks, with ankle support and anti-slip soles. Price: $149.00",
  "The Nomad Filter Bottle purifies water from natural sources and holds up to 1L of liquid. Price: $49.50",
  "All travel gear products come with a 2-year warranty and optional 24h delivery upgrade.",
  "The Essentials Travel Bundle includes a backpack, charger, and rain jacket at 15% off bundled price. Bundle Price: $215.00",
  "Each product supports customer reviews and is frequently recommended by outdoor travel influencers."
]

client = chromadb.Client()
collection = client.get_or_create_collection(name="retail_knowledge")

# Store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embed(model="mxbai-embed-large", input=d)
  embeddings = response["embeddings"]
  collection.add(
    ids=[str(i)],
    embeddings=embeddings,
    documents=[d]
  )


### 🔍 Querying the Vector Store with a Prompt

Now that we've stored our documents as embeddings, we can perform **semantic search** using a new input prompt.

This step simulates how an agent might retrieve relevant knowledge using **Retrieval-Augmented Generation (RAG)**.


In [ ]:
# an example input
prompt = "What animals are llamas related to?"

# generate an embedding for the input and retrieve the most relevant doc
response = ollama.embed(
  model="mxbai-embed-large",
  input=prompt
)
print (response)
results = collection.query(
  query_embeddings=response["embeddings"],
  n_results=1
)
data = results['documents'][0][0]
print (data)

# 🧠 Starting the CrewAI Section

##Now we define agents using CrewAI, connected to our locally running Ollama models.  
##This enables multi-agent workflows powered by lightweight, self-hosted LLMs.


In [ ]:
# @title 👨‍🦯 Run this cell to hide all warnings (optional)
# Warning control
import warnings
warnings.filterwarnings('ignore')

# To avoid the restart session warning in Colab, exclude the PIL and
# pydevd_plugins packages from being imported. This is fine because
# we didn't execute the code in the kernel session afterward.

# import sys
# sys.modules.pop('PIL', None)

In [ ]:
# @title ⬇️ Install project dependencies by running this cell
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewai_tools langchain_groq langchain_anthropic langchain_community cohere --quiet
print("---")
%pip show crewAI crewai_tools langchain_groq langchain_anthropic langchain_community cohere

## 🧩 Step 3: CrewAI Integration


In [ ]:
# imports
#from crewai import Agent, Task, Crew, Process
#from textwrap import dedent



## Define Agents
In CrewAI, agents are autonomous entities designed to perform specific roles and achieve particular goals. Each agent uses a language model (LLM) and may have specialized tools to help execute tasks.

In [ ]:
# @title 🕵🏻 Define your agents

from crewai import Agent
from textwrap import dedent
# Define LLM (OpenAI used here; replace as needed)
from crewai import LLM

llm = LLM(model="ollama/deepseek-r1:7b", base_url="http://127.0.0.1:11434")


In [ ]:
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)


In [ ]:
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [ ]:
agent_specs = {
    "SalesRep": {
        "role": "SalesRep",
        "goal": "Engage with customer inquiries and identify their needs.",
        "backstory": "Retail sales agent trained in conversational selling and client discovery.",
        "llm": "ollama/deepseek-r1:7b",
        "knowledge": [
      "Our store specializes in travel and outdoor gear. The most popular customer profiles are: hikers, digital nomads, and weekend travelers.",
      "Sales typically start with customer inquiries via chat, Telegram, or in-store. The SalesRep must open the conversation and guide the customer naturally.",
      "The SalesRep should ask qualifying questions like: 'What kind of trip are you planning?', 'Do you already own any of this gear?', and 'Are you traveling light or packing for longer stays?'",
      "First-time customers automatically receive a 10% discount on any order over $100.",
      "SalesReps should avoid recommending products directly — instead, they pass the customer's preferences to the ProductRecommender agent.",
      "Our communication style is friendly, conversational, and helpful — we prioritize rapport over hard selling.",
      "When ending a conversation, the SalesRep should pass the lead along with notes to the next agent, and optionally schedule a follow-up."
    ]
    },
    "ProductRecommender": {
        "role": "ProductRecommender",
        "goal": "Suggest suitable products based on customer preferences.",
        "backstory": "Expert in matching customer needs with available stock.",
        "llm": "ollama/deepseek-coder:6.7b",
        "knowledge": [
      "Available products and prices:",
      "- MountainFlow 45L Backpack – waterproof, lightweight, ideal for 2–5 day trips. Price: $89.99",
      "- CloudShell Rain Jacket – windproof, breathable, for unpredictable weather. Price: $129.00",
      "- PowerCore Mini Charger – 5200mAh, USB-C, compact and TSA-compliant. Price: $29.95",
      "- TrailMax Pro Hiking Shoes – ankle support, anti-slip sole. Price: $149.00",
      "- Nomad Filter Bottle – filters water, 1L capacity. Price: $49.50",
      "- Essentials Travel Bundle: includes backpack, charger, and rain jacket at 15% off. Bundle Price: $215.00",
      "Recommendation criteria:",
      "- Consider customer needs: weather, terrain, travel duration, weight preferences, and destination.",
      "- Recommend in groups: for example, backpack + charger + rain jacket.",
      "- Mention reviews and top-rated gear when relevant.",
      "- Use value language: 'lightweight', 'durable', 'fast-charging', 'waterproof', etc.",
      "All recommendations must be justified and mapped back to the customer's stated intent or preference."
    ]
    },
    "QuotationAgent": {
        "role": "QuotationAgent",
        "goal": "Prepare detailed and accurate price quotes.",
        "backstory": "Skilled in pricing logic, tax rules, and markdown strategies.",
        "llm": "ollama/deepseek-r1:7b",
        "knowledge": [
      "Quotes must include:",
      "- List of products with unit prices and quantities.",
      "- Subtotal before discounts.",
      "- Discounts (if applicable).",
      "- Tax (8% standard).",
      "- Shipping fee (if applicable).",
      "- Final total with everything included.",
      "Discount policy:",
      "- 10% off for new customers (first purchase only, over $100).",
      "- 15% off for Essentials Bundle (automatically applied).",
      "Shipping options:",
      "- Standard shipping (3–5 business days): free for orders over $150.",
      "- Express delivery (24 hours): flat rate of $19.99.",
      "Price references (USD):",
      "- Backpack: $89.99",
      "- Jacket: $129.00",
      "- Charger: $29.95",
      "- Shoes: $149.00",
      "- Bottle: $49.50",
      "- Bundle: $215.00",
      "All prices should be calculated to 2 decimal places. Output should be clean and suitable for PDF or email formatting.",
      "Mention the validity of the quote (default: 7 days) and include a reference ID."
    ]
    },
    "CRMUpdater": {
        "role": "CRMUpdater",
        "goal": "Log sales interactions and outcomes into the CRM.",
        "backstory": "Ensures customer data is updated after each interaction.",
        "llm": "ollama/deepseek-r1:7b",
        "knowledge": [
      "A CRM log should include the following fields:",
      "- Customer name",
      "- Contact method (Telegram handle, email)",
      "- Date and time of interaction",
      "- Products discussed or quoted (by title)",
      "- Quote total, discount applied",
      "- Preferred follow-up channel and time",
      "- Assigned sales agent or next responsible agent",
      "Example CRM entry:",
      "{ 'name': 'Elena Popa', 'contact': '@elenap', 'interested_products': ['CloudShell Rain Jacket', 'Nomad Filter Bottle'], 'quote_total': 160.55, 'discount': '10% new customer', 'followup_method': 'Telegram', 'followup_date': '2025-06-26' }",
      "CRM logs should use JSON format for easy integration with external systems.",
      "All entries must include a timestamp and agent signature for traceability.",
      "Follow-up reminders should be triggered if no action is taken within 24h of quote delivery.",
      "CRM data supports analytics for customer lifetime value, sales conversion rate, and product interest trends."
    ]
    }
}


In [ ]:
!pip install 'crewai[tools]'

In [ ]:
# ✅ Define your agents
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

_agents = []
_agent_names = []
for name, spec in agent_specs.items():
    _agent_names.append(name)
    _agents.append( Agent(
        role=spec["role"],
        goal=spec["goal"],
        backstory=spec["backstory"],
        knowledge_sources=[StringKnowledgeSource(
            content=str(spec["knowledge"])
        )],
        verbose=True,
        llm=LLM(model=spec["llm"], base_url="http://127.0.0.1:11434")
    ))



In [ ]:
# Test agent knowledge retrieval
agent = _agents[0]
if hasattr(agent, 'knowledge') and agent.knowledge:
    test_query = ["test query"]
    results = agent.knowledge.query(test_query)
    print(f"Agent knowledge results: {len(results)} documents found")

    # Test crew knowledge retrieval (if exists)
    if hasattr(crew, 'knowledge') and crew.knowledge:
        crew_results = crew.query_knowledge(test_query)
        print(f"Crew knowledge results: {len(crew_results)} documents found")

## Define Tasks
Tasks in CrewAI are specific assignments given to agents, detailing the actions they need to perform to achieve a particular goal. Tasks can have dependencies and context, and can be executed asynchronously to ensure an efficient workflow.

In [ ]:
from crewai import Task
from textwrap import dedent

task_data = [
    {
        "description": "Welcome a new customer who is planning a hiking trip. Ask engaging questions to understand their travel duration, weather expectations, and gear they already own.",
        "type": "conversation",
        "complexity": 2
    },
    {
        "description": "Based on the customer's hiking trip context, recommend a complete gear set including essentials like a backpack, outerwear, and accessories. Justify each suggestion.",
        "type": "recommendation",
        "complexity": 4
    },
    {
        "description": "Create a price quote for the recommended products. Apply a 10% first-time customer discount and calculate the final amount with 8% tax. Offer both standard and express shipping.",
        "type": "pricing",
        "complexity": 3
    },
    {
        "description": "Log the customer's contact info, travel plans, selected products, quote details, and preferred follow-up method into the CRM in structured JSON format.",
        "type": "data_entry",
        "complexity": 2
    },
    {
        "description": "Compose a personalized follow-up message for the customer, summarizing their quote and reminding them of the offer expiration and available delivery options.",
        "type": "followup",
        "complexity": 2
    }
]



In [ ]:
def get_response(prompt):
    url = "http://127.0.0.1:11434/v1/completions"
    headers = {"Content-Type": "application/json"}
    data = {
        "prompt": prompt,
        "model": "deepseek-r1:7b"
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            response_data = response.json()
            return response_data.get("choices", [{}])[0].get("text", "").strip()
        else:
            print(f"⚠️ Error: HTTP {response.status_code}")
            print(response.text)
            return "Planner"  # fallback to a valid agent name
    except Exception as e:
        print(f"❌ Request failed: {e}")
        return "PlannerBot"



In [ ]:
 agent_descriptions = "\n".join([
    f"{i+1}. {name}: {spec['goal']}" for i, (name, spec) in enumerate(agent_specs.items())
])
 print( agent_descriptions)


In [ ]:
def choose_best_agent(task, agents, llm_func):
    agent_descriptions = "\n".join([
    f"{i+1}. {name}: {spec['goal']}" for i, (name, spec) in enumerate(agent_specs.items())
])

    prompt = f"""
You are a task allocator AI.

Your job is to assign the following task to the most appropriate agent based on their expertise.

🔧 Agent's Goal:
\"{task['description']}\" (Type: {task['type']}, Complexity: {task['complexity']})

🧠 List of Avaliable Agents:
{agent_descriptions}

👉 Your task:
 Choose one agent from the List of Avaliable Agents, what fit the best with the goal. Respond only with the agent's **name**, exactly as written in th List of Avaliable Agents, this is very important.
The format of the response is one word - the agents name. No verbosity, just one name.
Example of response:
FirstAgent
"""
    print(prompt)
    response = llm_func(prompt).split("</think>")[1]

    return response.strip()


In [ ]:
from crewai import Task
from textwrap import dedent

task_objects = []
print (_agent_names)
for task in task_data:
    selected_name = choose_best_agent(task, agent_specs, get_response)
    print(selected_name)
    selected_agent = _agents[_agent_names.index(selected_name)]#.get(selected_name, list(agents.values())[0])  # fallback

    task_obj = Task(
        description=dedent(task["description"]),
        expected_output=dedent(f"Provide a full solution for: {task['description']}"),
        agent=selected_agent
    )
    task_objects.append(task_obj)
    print(f"✅ Assigned '{task['description']}' to {selected_name}")


In [ ]:
import os
from crewai.utilities.paths import db_storage_path

storage_path = db_storage_path()
print(f"Storage path: {storage_path}")
print(f"Path exists: {os.path.exists(storage_path)}")
print(f"Is writable: {os.access(storage_path, os.W_OK) if os.path.exists(storage_path) else 'Path does not exist'}")

# Create with proper permissions
if not os.path.exists(storage_path):
    os.makedirs(storage_path, mode=0o755, exist_ok=True)
    print(f"Created storage directory: {storage_path}")

In [ ]:
import chromadb
from crewai.utilities.paths import db_storage_path

# Connect to CrewAI's ChromaDB
storage_path = db_storage_path()
chroma_path = os.path.join(storage_path, "knowledge")

if os.path.exists(chroma_path):
    client = chromadb.PersistentClient(path=chroma_path)
    collections = client.list_collections()

    print("ChromaDB Collections:")
    for collection in collections:
        print(f"  - {collection.name}: {collection.count()} documents")
else:
    print("No ChromaDB storage found")

In [ ]:
from crewai import Crew, Process
from crewai import LLM

llm = LLM(model="ollama/deepseek-r1:7b", base_url="http://127.0.0.1:11434")
crew = Crew(
    agents=_agents,
    tasks=task_objects,
    memory=True,
    embedder={
        "provider": "ollama",
        "config": {"model": "mxbai-embed-large",
                   "url": "http://localhost:11434/api/embeddings"}
    },
    process=Process.sequential,
    llm=llm


)
#print(list(agents.values()))
result = crew.kickoff()
print("🧠 Final Result:\n", result)



In [ ]:
# NONONO Reset specific memory types
#crew.reset_memories(command_type='short')     # Short-term memory
#crew.reset_memories(command_type='long')      # Long-term memory
#crew.reset_memories(command_type='entity')    # Entity memory
#crew.reset_memories(command_type='knowledge') # Knowledge storage

In [ ]:
# @title 🖥️ Display the results of your crew as markdown
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))